In [1]:
  
import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

###############################################################################
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=(3,1),
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
  #2D Convolution-Batch Normalization-Activation stack builder   
   
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v1(input_shape, depth, num_filters=16, num_classes=10,
              nstack=3, kernel_size=(3,1), pool_size=(2,1)):
    
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 1) Conv2D-BN-ReLU
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    #num_filters = 16        
        
    num_res_blocks = int((depth - 2) / 6)

    inputs =Input(shape=input_shape)
	#inputs=input_shape
    x = resnet_layer(inputs=inputs, num_filters=num_filters,
                     kernel_size=kernel_size)
    # Instantiate the stack of residual units
    for stack in range(nstack):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,kernel_size=kernel_size,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,kernel_size=kernel_size,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2
        
            

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=pool_size)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='relu',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model
